# 1. [threading.Thread]启动与停止线程

`threading` 库可以在单独的线程中执行任何的在 `Python` 中可以调用的对象。你可以创建一个 `Thread` 对象并将你要执行的对象以 `target` 参数的形式提供给该对象

In [1]:
# Code to execute in an independent thread
import time
def countdown(n):
    while n > 0:
        print('T-minus', n)
        n -= 1
        time.sleep(5)

# Create and launch a thread
from threading import Thread
t = Thread(target=countdown, args=(10,))
t.start()

T-minus 10


当你创建好一个线程对象后，该对象并不会立即执行，除非你调用它的 `start()` 方法（当你调用 `start()` 方法时，它会调用你传递进来的函数，并把你传递进来的参数传递给该函数）。

`Python` 中的线程会在一个单独的系统级线程中执行（比如说一个 `POSIX` 线程或者一个 `Windows` 线程），这些线程将由操作系统来全权管理。

线程一旦启动，将独立执行直到目标函数返回。你可以查询一个线程对象的状态，看它是否还在执行：

```py
if t.is_alive():
    print('Still running')
else:
    print('Completed')

```

你也可以将一个线程加入到当前线程，并等待它终止：

```py
t.join()

```


后台线程无法等待，不过，这些线程会在主线程终止时自动销毁。 

除了如上所示的两个操作，并没有太多可以对线程做的事情。

你无法结束一个线程，无法给它发送信号，无法调整它的调度，也无法执行其他高级操作。

如果需要这些特性，你需要自己添加。

比如说，如果你需要终止线程，那么这个线程必须通过编程在某个特定点轮询来退出。

你可以像下边这样把线程放入一个类中：

In [23]:
%%file ThreadTest.py
import time
class CountdownTask:
    def __init__(self):
        self._running = True

    def terminate(self):
        self._running = False

    def run(self, n):
        while self._running and n > 0:
            print('T-minus', n)
            n -= 1
            time.sleep(5)

c = CountdownTask()

from threading import Thread
t = Thread(target=c.run, args=(10,))
t.start()
c.terminate() # Signal termination 信號終止
t.join()      # Wait for actual termination (if needed) 實際停止

Overwriting ThreadTest.py


In [22]:
# ============ 開新 CONSOLE ============
# ------------ Run the server ------------

import os
import subprocess
# os.path.abspath {本黨位置}: D:\Google 雲端硬碟\learn\線程調用\TestOS.py
# os.path.dirname {目錄} : D:\Google 雲端硬碟\learn\線程調用
BASE_DIR = os.path.dirname(os.path.abspath('__file__'))
# 透過 cmd 呼叫
DIR = os.path.join(BASE_DIR, 'ThreadTest.py')
cmd = "python " + f'"{DIR}"'
print(cmd,BASE_DIR,sep='\n')
#  CONSOLE混雜
#os.system(cmd)
#subprocess.call(cmd)

#  NEW 一個 CONSOLE
subprocess.Popen(cmd, creationflags=subprocess.CREATE_NEW_CONSOLE)

python "D:\CODE\GitHub\py\資料結構\py3-cookbook\并发编程\ThreadTest.py"
D:\CODE\GitHub\py\資料結構\py3-cookbook\并发编程


如果线程执行一些像 `I/O` 这样的阻塞操作，那么通过轮询来终止线程将使得线程之间的协调变得非常棘手。

比如，如果一个线程一直阻塞在一个 `I/O` 操作上，它就永远无法返回，也就无法检查自己是否已经被结束了。

要正确处理这些问题，你需要利用超时循环来小心操作线程。 例子如下：

```py
class IOTask:
    def terminate(self):
        self._running = False

    def run(self, sock):
        # sock is a socket
        sock.settimeout(5)        # Set timeout period
        while self._running:
            # Perform a blocking I/O operation w/ timeout
            try:
                data = sock.recv(8192)
                break
            except socket.timeout:
                continue
            # Continued processing
            ...
        # Terminated
        return
```



## 讨论

由于全局解释锁 `（GIL）` 的原因， `Python` 的线程被限制到同一时刻只允许一个线程执行这样一个执行模型。

所以， `Python` 的线程更适用于处理 `I/O` 和其他需要并发执行的阻塞操作（比如等待 `I/O` 、等待从数据库获取数据等等），而**不是需要多处理器并行的计算密集型任务**

你可以通过 `multiprocessing` 模块在一个单独的进程中执行你的代码：

In [18]:
import multiprocessing

class CountdownTask:
    def __init__(self):
        self._running = True

    def terminate(self):
        self._running = False

    def run(self, n):
        while self._running and n > 0:
            print('T-minus', n)
            n -= 1
            time.sleep(1)

c = CountdownTask()
p = multiprocessing.Process(
    target=c.run(n=5)
)
p.start()

T-minus 5
T-minus 4
T-minus 3
T-minus 2
T-minus 1


# [threading.Event()]判断线程是否已经启动

线程的一个关键特性是每个线程都是独立运行且状态不可预测。

如果程序中的其他线程需要通过判断某个线程的状态来确定自己下一步的操作，这时线程同步问题就会变得非常棘手。

为了解决这些问题，我们需要使用 `threading` 库中的 `Event` 对象。 

`Event` 对象包含一个可由线程设置的信号标志，它允许线程等待某些事件的发生。

在初始情况下， `event` 对象中的信号标志被设置为假。

如果有线程等待一个 `event` 对象，而这个 `event` 对象的标志为假，那么这个线程将会被一直阻塞直至该标志为真。

一个线程如果将一个 `event` 对象的信号标志设置为真，它将唤醒所有等待这个 `event` 对象的线程。

如果一个线程等待一个已经被设置为真的 `event` 对象，那么它将忽略这个事件，继续执行。 

下边的代码展示了如何使用 `Event` 来协调线程的启动：

In [19]:
from threading import Thread, Event
import time

# Code to execute in an independent thread
def countdown(n, started_evt):
    print('countdown starting')
    # Event().set()
    started_evt.set()
    while n > 0:
        print('T-minus', n)
        n -= 1
        time.sleep(5)

# Create the event object that will be used to signal startup
started_evt = Event()

# Launch the thread and pass the startup event
print('Launching countdown')
t = Thread(target=countdown, args=(10,started_evt))
t.start()

# Wait for the thread to start
started_evt.wait()
print('countdown is running')

Launching countdown
countdown starting
T-minus 10


True

countdown is running


## [threading.Condition()]讨论

`event` 对象最好单次使用，就是说，你创建一个 `event` 对象，让某个线程等待这个对象，一旦这个对象被设置为真，你就应该丢弃它。

尽管可以通过 `clear()` 方法来重置 `event` 对象，但是很难确保安全地清理 `event` 对象并对它重新赋值。

很可能会发生错过事件、死锁或者其他问题（特别是，你无法保证重置 `event` 对象的代码会在线程再次等待这个 `event` 对象之前执行）。

如果一个线程需要不停地重复使用 `event` 对象，你最好使用 `Condition` 对象来代替。

下面的代码使用 `Condition` 对象实现了一个周期定时器，每当定时器超时的时候，其他线程都可以监测到：

In [25]:
%%file MultiThreadTest.py
import threading
import time

# ======= Timer =======

class PeriodicTimer:
    def __init__(self, interval):
        self._interval = interval
        self._flag = 0
        self._cv = threading.Condition()

    def start(self):
        t = threading.Thread(target=self.run)
        t.daemon = True

        t.start()

    def run(self):
        '''
        Run the timer and notify waiting threads after each interval
        '''
        while True:
            time.sleep(self._interval)
            with self._cv:
                # 二進制 異位(XOR)
                self._flag ^= 1
                self._cv.notify_all()

    def wait_for_tick(self):
        '''
        Wait for the next tick of the timer
        '''
        with self._cv:
            last_flag = self._flag
            while last_flag == self._flag:
                self._cv.wait()

# Example use of the timer
ptimer = PeriodicTimer(1)
ptimer.start()

# Two threads that synchronize on the timer
def countdown(nticks):
    while nticks > 0:
        ptimer.wait_for_tick()
        print('T-minus', nticks)
        nticks -= 1

def countup(last):
    n = 0
    while n < last:
        ptimer.wait_for_tick()
        print('Counting', n)
        n += 1

threading.Thread(target=countdown, args=(10,)).start()
threading.Thread(target=countup, args=(5,)).start()

Overwriting MultiThreadTest.py


In [26]:
# ============ 開新 CONSOLE ============
# ------------ Run the server ------------

import os
import subprocess
# os.path.abspath {本黨位置}: D:\Google 雲端硬碟\learn\線程調用\TestOS.py
# os.path.dirname {目錄} : D:\Google 雲端硬碟\learn\線程調用
BASE_DIR = os.path.dirname(os.path.abspath('__file__'))
# 透過 cmd 呼叫
DIR = os.path.join(BASE_DIR, 'MultiThreadTest.py')
cmd = "python " + f'"{DIR}"'
print(cmd,BASE_DIR,sep='\n')
#  CONSOLE混雜
#os.system(cmd)
#subprocess.call(cmd)

#  NEW 一個 CONSOLE
subprocess.Popen(cmd, creationflags=subprocess.CREATE_NEW_CONSOLE)

python "D:\CODE\GitHub\py\資料結構\py3-cookbook\并发编程\MultiThreadTest.py"
D:\CODE\GitHub\py\資料結構\py3-cookbook\并发编程


`event` 对象的一个重要特点是当它被设置为真时会唤醒所有等待它的线程。

如果你只想唤醒单个线程，最好是使用 **信号量** 或者 `Condition` 对象来替代。考虑一下这段使用信号量实现的代码：

In [42]:
import threading
# Worker thread
def worker(n, sema):
    # Wait to be signaled
    sema.acquire()

    # Do some work
    print('Working', n)

# Create some threads
sema = threading.Semaphore(0)
nworkers = 10
for n in range(nworkers):
    t = threading.Thread(target=worker, args=(n, sema,))
    t.start()

每次信号量被释放，只有一个线程会被唤醒并执行

In [43]:
sema.release()

sema.release()

Working 0
Working 1


# 线程间通信

从一个线程向另一个线程发送数据最安全的方式可能就是使用 `queue` 库中的队列了。

创建一个被多个线程共享的 `Queue` 对象，这些线程通过使用 `put()` 和 `get()` 操作来向队列中添加或者删除元素。 例如：

In [44]:
from queue import Queue
from threading import Thread

# A thread that produces data
def producer(out_q):
    while True:
        # Produce some data
        ...
        out_q.put(data)

# A thread that consumes data
def consumer(in_q):
    while True:
        # Get some data
        data = in_q.get()
        # Process the data
        ...

# Create the shared queue and launch both threads
q = Queue()
t1 = Thread(target=consumer, args=(q,))
t2 = Thread(target=producer, args=(q,))
t1.start()
t2.start()

`Queue` 对象已经包含了必要的锁，所以你可以通过它在多个线程间多安全地共享数据。 

当使用队列时，协调生产者和消费者的关闭问题可能会有一些麻烦。

一个通用的解决方法是在队列中放置一个特殊的值，当消费者读到这个值的时候，终止执行。例如：

```py
from queue import Queue
from threading import Thread

# Object that signals shutdown
_sentinel = object()

# A thread that produces data
def producer(out_q):
    while running:
        # Produce some data
        ...
        out_q.put(data)

    # Put the sentinel on the queue to indicate completion
    out_q.put(_sentinel)

# A thread that consumes data
def consumer(in_q):
    while True:
        # Get some data
        data = in_q.get()

        # Check for termination
        if data is _sentinel:
            in_q.put(_sentinel)
            break

        # Process the data
        ...
```



In [46]:
from queue import Queue
from threading import Thread

# A thread that produces data
def producer(out_q):
    while True:
        # Produce some data
        ...
        out_q.put(data)

# A thread that consumes data
def consumer(in_q):
    while True:
        # Get some data
        data = in_q.get()

        # Process the data
        ...
        # Indicate completion
        in_q.task_done()

# Create the shared queue and launch both threads
q = Queue()
t1 = Thread(target=consumer, args=(q,))
t2 = Thread(target=producer, args=(q,))
t1.start()
t2.start()

# Wait for all produced items to be consumed
q.join()

Exception in thread Thread-39:
Traceback (most recent call last):
  File "D:\anaconda\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "D:\anaconda\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-46-42e697c95608>", line 9, in producer
    out_q.put(data)
NameError: name 'data' is not defined

